# Running hyperparameter optimization on Chemprop model using RayTune

## Import packages

In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

In [2]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['lipo'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

## Load data

In [3]:
df_input = pd.read_csv(input_path)
df_input

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


In [4]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

## Make data points, splits, and datasets

In [5]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [6]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

In [7]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)


# Define helper function to train the model

In [8]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSEMetric(), nn.metrics.MAEMetric()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(find_unused_parameters=True),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)


## Define parameter search space

In [9]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [10]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=True,
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch() 

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    
)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()


(RayTrainWorker pid=18448) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=18351) Started distributed worker processes: 
(TorchTrainer pid=18351) - (ip=10.114.0.73, pid=18448) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=18448) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=18448) GPU available: True (cuda), used: True
(RayTrainWorker pid=18448) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=18448) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=18448) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=18448) /home/hwpang/miniforge3/envs/chemprop_v2_dev/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` l

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]


(raylet) [2024-06-05 11:25:24,245 E 14850 14879] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-05_11-25-12_802597_14540 is over 95% full, available space: 99562876928; capacity: 2002947665920. Object creation will fail if spilling is required.
(RayTrainWorker pid=18682) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=18448) /home/hwpang/miniforge3/envs/chemprop_v2_dev/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('val/rmse', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=18448) /home/hwpang/miniforge3/envs/chemprop_v2_dev/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('val/mae', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across d

Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s]                             


(TorchTrainer pid=18449) Started distributed worker processes: 
(TorchTrainer pid=18449) - (ip=10.114.0.73, pid=18682) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=18682) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=18682) 2 | bn              | BatchNorm1d        | 800   
(RayTrainWorker pid=18682) GPU available: True (cuda), used: True
(RayTrainWorker pid=18682) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=18682) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=18682) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=18682) /home/hwpang/miniforge3/envs/chemprop_v2_dev/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: sr

...

2024-06-05 11:25:32,471	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).


Epoch 19: 100%|██████████| 2/2 [00:00<00:00, 32.03it/s, v_num=0, train_loss=0.164, val_loss=0.880] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|██████████| 2/2 [00:00<00:00, 10.31it/s, v_num=0, train_loss=0.164, val_loss=0.869]


(RayTrainWorker pid=18682) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14/TorchTrainer_bdfa8a4b_2_depth=2,ffn_hidden_dim=2200,ffn_num_layers=2,message_hidden_dim=400_2024-06-05_11-25-17/checkpoint_000019)
(RayTrainWorker pid=18682) `Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 2/2 [00:00<00:00,  7.20it/s, v_num=0, train_loss=0.164, val_loss=0.869]


2024-06-05 11:25:33,705	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-06-05 11:25:33,706	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14' in 0.0027s.
2024-06-05 11:25:33,709	ERROR tune.py:1044 -- Trials did not complete: [TorchTrainer_c8cd4ba9]
2024-06-05 11:25:33,709	INFO tune.py:1048 -- Total run time: 18.92 seconds (18.90 

## Hyperparameter optimization results

In [11]:
results

ResultGrid<[
  Result(
    error='RayTaskError(OutOfMemoryError)',
    metrics={},
    path='/home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14/TorchTrainer_c8cd4ba9_1_depth=2,ffn_hidden_dim=2000,ffn_num_layers=2,message_hidden_dim=500_2024-06-05_11-25-14',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'train_loss': 0.16392788290977478, 'val/rmse': 0.8689340353012085, 'val/mae': 0.7063738107681274, 'val_loss': 0.8689340353012085, 'epoch': 19, 'step': 40},
    path='/home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14/TorchTrainer_bdfa8a4b_2_depth=2,ffn_hidden_dim=2200,ffn_num_layers=2,message_hidden_dim=400_2024-06-05_11-25-17',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14/TorchTrainer_bdfa8a4b_2_depth=2,ffn_hidden_dim=

In [12]:
# results of all trials
result_df = results.get_dataframe()
result_df

,train_loss,val/rmse,val/mae,val_loss,epoch,step,timestamp,checkpoint_dir_name,should_checkpoint,done,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.163928,0.868934,0.706374,0.868934,19,40,1717601132,checkpoint_000019,True,False,...,18449,estes,10.114.0.73,12.113055,20,2,2200,2,400,bdfa8a4b


In [13]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [14]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /home/hwpang/Projects/chemprop_v2_dev/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-06-05_11-25-14/TorchTrainer_bdfa8a4b_2_depth=2,ffn_hidden_dim=2200,ffn_num_layers=2,message_hidden_dim=400_2024-06-05_11-25-17/checkpoint_000019/checkpoint.ckpt


In [15]:
ray.shutdown()